In [5]:
# use manually annotated labels

data_path = "../data/"
traces_path = "5_ode.csv"

import pandas as pd
df = pd.read_csv(data_path+traces_path)
print(df.head(5))
# print(df['New'])

   case ID resource     activity name   resource+activity     Sitter      New  \
0        1        A        greeting()        A|greeting()   A|hello*    Hello   
1        1        U    question(data)    U|question(data)  U|request    Query   
2        1        A    list(keywords)    A|list(keywords)    A|offer  Request   
3        1        U     set(keywords)     U|set(keywords)  U|request    Query   
4        1        A  prompt(keywords)  A|prompt(keywords)    A|offer  Request   

     New.1  
0    Hello  
1    Query  
2  Request  
3    Query  
4  Request  


In [1]:
# collect dialogues
case = 0
dialogues = []
for index, utterance in df.iterrows():
    if utterance['case ID'] != case:
        # use start and end symbols
        if dialogues:
            dialogues[-1] += '>'
        dialogues.append('<')
        case = utterance['case ID']
    if utterance['New'] != 'Hello':
        dialogues[-1] += utterance['New'][0]
dialogues[-1] += '>'
print(len(dialogues))
print(dialogues)

26
['<QRQRQRQRQAF>', '<RQRFAQRQRQAF>', '<QRQRQAF>', '<QAQRQRQRQAF>', '<QRQRQRQAQRQRF>', '<QRQRQAFRFRQRQAQRQRFRFRQAFAFA>', '<QRQAQRQRFRF>', '<RQRQRQAQAFRFAQRFRF>', '<RRQAQRQAFRQAQRQA>', '<QQRQRQRQRQRFRQRFRFRFRQRQRQRF>', '<QRFRFRQRFRQAFRFRQRFRFRQRQRQRQRQRQRQA>', '<QAFRF>', '<QRQRQRQRQRFRFRFRFRQRQRFAQRFRF>', '<RQRQAQAFAFAFRFAFA>', '<QRQRFRQRFRFA>', '<QRQRQAFRF>', '<QAQAAF>', '<QRFRFRFRQRFRFRFRQRFRQAFRQRQAFA>', '<QRQAQRFAFA>', '<RQRQRQAQAF>', '<RQAQAF>', '<QRQAQRQAFRFRFRFAF>', '<RQRQRQRFRQAF>', '<QRQRQAFRQAQAFAQRFRQRFAFAFA>', '<QAQAQAFRFAF>', '<QRFAFA>']


In [3]:
# calculate number of answers per dialogue -- information presentation
for d in dialogues:
    print(Counter(d)['A'])

1
2
1
2
1
5
1
3
4
0
2
1
1
6
1
1
3
3
3
2
2
3
1
7
4
2


In [2]:
# 1) count all bigrams
from collections import Counter

def find_ngrams(dialogues, n=2):
    # mine ngrams with a sliding window n (linear), e.g. QR, RQ
    ngrams = Counter()
    s = ''
    for d in 'o'.join(dialogues):
        if d == 'o':
            # reset pattern
            s = ''
            continue
        # skip duplicate symbols (loops)
        if not s or s[-1] != d:
            s += d
        if len(s) == n:
            ngrams[s] += 1
            # slide over
            s = s[1:]
    return ngrams

ngrams = find_ngrams(dialogues)
# absolute counts for bigrams
print(list(ngrams.values()))
_max = max(list(ngrams.values()))
ngrams = {k: round(v / _max, 2) for k, v in ngrams.items()}
print(ngrams)

# absolute counts for unigrams
# ngrams = find_ngrams(dialogues, n=1)
# print(ngrams.most_common())
# # normalise counts to be relative to the total number of conver
# das = {k: v / total for k, v in ngrams.items() if k in ['Q', 'R', 'F', 'A']}
# total = sum(das.values(), 0.0)
# ngrams = {k: round(v / total, 2) for k, v in das.items()}
# print(ngrams)

[19, 80, 81, 40, 32, 17, 7, 50, 21, 20, 44, 9]
{'<Q': 0.23, 'QR': 0.99, 'RQ': 1.0, 'QA': 0.49, 'AF': 0.4, 'F>': 0.21, '<R': 0.09, 'RF': 0.62, 'FA': 0.26, 'AQ': 0.25, 'FR': 0.54, 'A>': 0.11}


In [32]:
# count only one occurance of the same pattern per conversation
# 1) count all bigrams
from collections import Counter

def find_ngrams(dialogues, n=2):
    # mine ngrams with a sliding window n (linear), e.g. QR, RQ
    ngrams = Counter()
    s = ''
    c_patterns = []
    for d in 'o'.join(dialogues):
        # keep track of the patterns in the same conversation
        if d == 'o':
            # reset pattern
            s = ''
            c_patterns = []
            continue
        # skip duplicate symbols (loops)
        if not s or s[-1] != d:
            s += d
        if len(s) == n:
            # count only once per conversation
            if s not in c_patterns:
                ngrams[s] += 1
                c_patterns.append(s)
            # slide over
            s = s[1:]
    return ngrams

ngrams = find_ngrams(dialogues)
# absolute counts for bigrams
# print(ngrams.most_common())
# normalise counts to be relative to the total number of conver
total = 26
ngrams = {k: round(v / total, 2) for k, v in ngrams.items()}
print(ngrams)

{'<Q': 0.73, 'QR': 0.85, 'RQ': 0.85, 'QA': 0.85, 'AF': 0.81, 'F>': 0.65, '<R': 0.27, 'RF': 0.73, 'FA': 0.46, 'AQ': 0.62, 'FR': 0.62, 'A>': 0.35}


In [ ]:
# evaluate against manually annotated labels


In [ ]:
# 1) count all ngrams


In [ ]:
# 1st transformation
def find_loops(dialogues):
    # mine loops (linear), e.g. QRQ, i.e. s[0] == s[2] -> s[0:1] is a loop
    loops = []
    s = ''
    for d in 'o'.join(dialogues):
        if d == 'o':
            # reset pattern
            s = ''
            continue
        s += d
        if len(s) == 3:
            # check if loop
            if s[0] == s[2]:
                loop = s[:2]
                if loop not in loops:
                    loops.append(loop)
            # slide over
            s = s[1:]
    return loops

loops = find_loops(dialogues)
print(loops)